## Setup

**Import Statements**

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep
from sklearn.metrics import r2_score
from amb_sdk.sdk import DarwinSdk

**Connection to Darwin**

In [2]:
# Login
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('mprost@utexas.edu', 'FbSnZ663sf')

if not status:
    print(msg)


**Setup Path**

In [3]:
path = './sets/'

## Upload and Read Data

**Read Data**

In [4]:
dataset_name = 'remittances_data_q2.csv'
df = pd.read_csv(os.path.join(path, dataset_name))
df = df.drop(columns=['id', 'firm', 'cc1 lcu amount', 'cc1 denomination amount', 
                      'cc2 lcu amount', 'cc2 denomination amount', 
                      'Standard Note', 'note2', 'date'])
df.to_csv('./sets/remittances_data_q2__tcp_cleaned3.csv', index=False)
dataset_name = 'remittances_data_q2__tcp_cleaned3.csv'
df.head()

/Users/Matt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,period,source_code,source_name,source_region,source_income,source_lending,source_G8G20,destination_code,destination_name,destination_region,...,cc2 lcu code,cc2 lcu fee,cc2 lcu fx rate,cc2 fx margin,cc2 total cost %,inter lcu bank fx,transparent,receiving network coverage,pickup method,corridor
0,2016_2Q,AGO,Angola,Sub-Saharan Africa,Lower middle income,IBRD,..,NAM,Namibia,Sub-Saharan Africa,...,AOA,5700.0,0.08,7.49,14.36,0.09,yes,High,Agent,AGONAM
1,2016_2Q,AGO,Angola,Sub-Saharan Africa,Lower middle income,IBRD,..,NAM,Namibia,Sub-Saharan Africa,...,AOA,6300.0,0.09,4.19,11.78,0.09,yes,High,Agent,AGONAM
2,2016_2Q,AGO,Angola,Sub-Saharan Africa,Lower middle income,IBRD,..,NAM,Namibia,Sub-Saharan Africa,...,AOA,9075.0,1.00,0.00,10.93,1.00,no,Low,NaN,AGONAM
3,2016_2Q,ARE,United Arab Emirates,Middle East & North Africa,High income,..,..,BGD,Bangladesh,South Asia,...,AED,20.0,21.32,0.08,1.17,21.34,yes,High,Agent,AREBGD
4,2016_2Q,ARE,United Arab Emirates,Middle East & North Africa,High income,..,..,BGD,Bangladesh,South Asia,...,AED,15.0,21.31,0.13,0.95,21.34,yes,High,Agent,AREBGD


**Upload to Darwin**

In [5]:
# Upload dataset
status, message = ds.delete_dataset(dataset_name)
status, dataset = ds.upload_dataset(os.path.join(path, dataset_name))
if not status:
    print(dataset)

## Clean Data

In [6]:
# clean dataset
target = "cc1 total cost %"
status, job_id = ds.clean_data(dataset_name, target=target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)
    


{'status': 'Requested', 'starttime': '2019-04-22T17:14:44.64079', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': ['efae012e294746289b1d00d1be24cad4'], 'model_name': None, 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:14:44.64079', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': ['efae012e294746289b1d00d1be24cad4'], 'model_name': None, 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:14:44.64079', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': ['efae012e294746289b1d00d1be24cad4'], 'model_name': None, 'job_error': None}
{'status': 'Requested', '

{'status': 'Requested', 'starttime': '2019-04-22T17:14:44.64079', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': ['efae012e294746289b1d00d1be24cad4'], 'model_name': None, 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:14:44.64079', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': ['efae012e294746289b1d00d1be24cad4'], 'model_name': None, 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:14:44.64079', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': ['efae012e294746289b1d00d1be24cad4'], 'model_name': None, 'job_error': None}
{'status': 'Requested', '

## Create and Train Model

In [9]:
model = target + "_model5"
ds.delete_model(model)
status, job_id = ds.create_model(dataset_names = dataset_name, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model5', 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model5', 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model5', 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endti

{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model5', 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model5', 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['remittances_data_q2__tcp_cleaned3.csv'], 'artifact_names': None, 'model_name': 'cc1 total cost %_model5', 'job_error': None}
{'status': 'Requested', 'starttime': '2019-04-22T17:37:04.344209', 'endti

In [10]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

404: NOT FOUND - {"message": "Failed to find a dataset that the model was trained on. You have requested this URI [/v1/analyze/model/cc1 total cost %_model5] but did you mean /v1/analyze/model/<model_name> ?"}



TypeError: string indices must be integers

In [ ]:
feature_importance[:10]

## Analyze Data

**Model Prediction**

In [ ]:
#print (dataset_name)
#print (model)
status, artifact = ds.run_model(dataset_name, model)
#print (status)
#print (run_id)
sleep(1)
ds.wait_for_job(artifact['job_name'])

**Get Predictions from Darwin**

In [ ]:
status, prediction = ds.download_artifact(artifact['artifact_name'])
prediction.head()

In [ ]:
#Plot predictions vs actual
print(len(prediction[target]))

plt.plot(df[target], prediction[target], '.')
plt.plot([0,2.3e7],[0,2.3e7],'--k')
plt.show()
print('R^2 : ', r2_score(df[target], prediction[target]))